This is an NLP modeling exercise (**Char-RNN**) followed the instruction of [*Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow*](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) on topic **Natural Language Processing with RNNs and Attention**.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Create the training dataset

## Download Shakespeare's work

In [ ]:
shakespeare_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
  shakespeare_txt = f.read()

1130496/1115394 [==============================] - 0s 0us/step


## Encode every character as an interger

In [ ]:
# Encode on character level
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_txt])

In [ ]:
# Check word_index map dictionary
word_index=tokenizer.word_index
word_index

{'\n': 11,
 ' ': 1,
 '!': 31,
 '$': 39,
 '&': 38,
 "'": 28,
 ',': 18,
 '-': 32,
 '.': 27,
 '3': 37,
 ':': 24,
 ';': 29,
 '?': 30,
 'a': 5,
 'b': 22,
 'c': 19,
 'd': 13,
 'e': 2,
 'f': 20,
 'g': 21,
 'h': 7,
 'i': 6,
 'j': 33,
 'k': 25,
 'l': 12,
 'm': 15,
 'n': 10,
 'o': 4,
 'p': 23,
 'q': 34,
 'r': 9,
 's': 8,
 't': 3,
 'u': 14,
 'v': 26,
 'w': 17,
 'x': 35,
 'y': 16,
 'z': 36}

In [ ]:
# Get the number of unique character IDs
max_id = len(tokenizer.word_index)
max_id

39

In [ ]:
# Test on a sample
sample = 'First'
encoded_sample = tokenizer.texts_to_sequences([sample])
encoded_sample

[[20, 6, 9, 8, 3]]

In [ ]:
decoded_sample = tokenizer.sequences_to_texts(encoded_sample)
decoded_sample

['f i r s t']

In [ ]:
# Encode the whole text
[encoded_text] = np.array(tokenizer.texts_to_sequences([shakespeare_txt])) - 1
encoded_text

array([19,  5,  8, ..., 20, 26, 10])

## Split sequential dataset

In [ ]:
# Train test split
dataset_size = len(encoded_text)
split_size = int(dataset_size * 0.9)
train_dataset = tf.data.Dataset.from_tensor_slices(encoded_text[:split_size])
test_dataset = tf.data.Dataset.from_tensor_slices(encoded_text[split_size:])

train_dataset, test_dataset

(<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>,
 <TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>)

## Chop sequential dataset into windows

In [ ]:
N_STEPS = 100
SHIFT = 1

window_length = N_STEPS + SHIFT
train_dataset = train_dataset.window(window_length, shift=SHIFT, drop_remainder=True)
test_dataset = test_dataset.window(window_length, shift=SHIFT, drop_remainder=True)

train_dataset, test_dataset

(<WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([]))>,
 <WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([]))>)

In [ ]:
# Convert nested dataset into flat dataset
train_dataset = train_dataset.flat_map(lambda x: x.batch(window_length))
test_dataset = test_dataset.flat_map(lambda x: x.batch(window_length))

In [ ]:
# Process dataset into features and labels
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(10000).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

train_dataset = train_dataset.map(lambda x: (x[:, :-1], x[:, 1:]))
test_dataset = test_dataset.map(lambda x: (x[:, :-1], x[:, 1:]))

train_dataset, test_dataset

(<MapDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>,
 <MapDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>)

In [ ]:
# One-hot encode features
train_dataset = train_dataset.map(lambda x, y: (tf.one_hot(x, depth=max_id), y))
test_dataset = test_dataset.map(lambda x, y: (tf.one_hot(x, depth=max_id), y))

train_dataset, test_dataset

(<MapDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>,
 <MapDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>)

In [ ]:
# Configure for better performance
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>)

In [ ]:
# Get a view of the processed dataset
for X_batch, Y_batch in train_dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [ ]:
len(encoded_text)/32 *0.1

3485.6062500000003

# Built and train Char-RNN model

In [ ]:
# Construct model
model = tf.keras.Sequential([layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
                             layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
                             layers.TimeDistributed(layers.Dense(max_id, activation='softmax'))])

# Compile model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit model 
# (due to the large size of dataset and limited computaion power of my hardware, only a small portion of sataset is used in fitting process)
history = model.fit(train_dataset,
                   steps_per_epoch=100,
                   epochs=20,
                   validation_data=test_dataset,
                   validation_steps=20)

Epoch 1/20
100/100 [==============================] - 58s 506ms/step - loss: 2.9714 - accuracy: 0.1882 - val_loss: 2.6626 - val_accuracy: 0.2330
Epoch 2/20
100/100 [==============================] - 52s 518ms/step - loss: 2.4388 - accuracy: 0.3010 - val_loss: 2.4758 - val_accuracy: 0.2789
Epoch 3/20
100/100 [==============================] - 50s 497ms/step - loss: 2.3219 - accuracy: 0.3241 - val_loss: 2.3678 - val_accuracy: 0.2992
Epoch 4/20
100/100 [==============================] - 50s 499ms/step - loss: 2.1737 - accuracy: 0.3590 - val_loss: 2.2933 - val_accuracy: 0.3234
Epoch 5/20
100/100 [==============================] - 49s 489ms/step - loss: 2.0634 - accuracy: 0.3887 - val_loss: 2.2516 - val_accuracy: 0.3339
Epoch 6/20
100/100 [==============================] - 50s 503ms/step - loss: 1.9817 - accuracy: 0.4067 - val_loss: 2.2128 - val_accuracy: 0.3507
Epoch 7/20
100/100 [==============================] - 50s 499ms/step - loss: 1.9236 - accuracy: 0.4213 - val_loss: 2.1915 - val_ac

# Use the trained model to predict

In [ ]:
# Define a function to preprocess text
def preprocess(text):
  X = np.array(tokenizer.texts_to_sequences(text)) - 1
  return tf.one_hot(X, max_id)

In [ ]:
# Predict the next letter
X_encoded = preprocess(['How are yo'])
y_pred = model.predict(X_encoded)
y_pred = y_pred[0].argmax(axis=1)
y_pred_decoded = tokenizer.sequences_to_texts(np.expand_dims(y_pred, axis=0) + 1)[0][-1]
y_pred_decoded

'u'

# Generate fake Shakespeare text

In [ ]:
# Create a function to pick the next character randomly
def next_char(text, temperature):
  X_encoded = preprocess([text])
  y_proba = model.predict(X_encoded)[0,-1:,:]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
# Write a function to generate characters consecutively
def complete_text(text, n_chars, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

In [ ]:
# Text the model's predictive power
print(complete_text('She is ugl', n_chars=50, temperature=1))

She is ugle eser to preantly,
a pereifure an canure number f


In [ ]:
print(complete_text('A pig is flying w', n_chars=50, temperature=2))

A pig is flying witjenkat; quusproces and veigre.

all cmxic? you,



In [ ]:
print(complete_text('A pig is flying w', n_chars=50, temperature=0.5))

A pig is flying were such of the people,
the people, the people, an
